# Left, inner, outer and cross
By the end of this lecture you will be able to:
- left joins between `DataFrames`
- inner joins between `DataFrames`
- outer and cross joins between `DataFrames`

# 左连接、内连接、外连接和交叉连接

本讲结束时，您将能够：

- 数据框之间的左连接

- 数据框之间的内连接

- 数据框之间的外连接和交叉连接


In [2]:
import polars as pl

In [3]:
df1 = pl.DataFrame({"id": [1, 2, 3], "name": ["Alice", "Bob", "Charlie"]})
df1

id,name
i64,str
1,"""Alice"""
2,"""Bob"""
3,"""Charlie"""


In [4]:
df2 = pl.DataFrame({"id": [2, 3, 4], "age": [25, 30, 35]})
df2

id,age
i64,i64
2,25
3,30
4,35


### Inner Join:

    Combines rows from both DataFrames where the id column matches in both tables.
    Only rows with matching id values (2 and 3) are included in the result.
### 内连接：

合并两个 DataFrame 中 id 列值匹配的行。

结果中仅包含 id 值匹配（2 和 3）的行。

In [4]:
result = df1.join(df2, on="id", how="inner")

In [5]:
result

id,name,age
i64,str,i64
2,"""Bob""",25
3,"""Charlie""",30


## Left join
In a left join we return all the rows from the left `DataFrame` and the matched rows from the right `DataFrame`. The matching happens on the basis of the join column(s). If the join column name is the same in the left and right `DataFrames` we can pass it to the `on` argument

## 左连接

左连接会返回左侧 `DataFrame` 中的所有行以及右侧 `DataFrame` 中匹配的行。匹配基于连接列。如果左右 `DataFrame` 中的连接列名称相同，我们可以将其传递给 `on` 参数。

In [7]:
(df1.join(df2, on="id", how="left", coalesce=False))

id,name,id_right,age
i64,str,i64,i64
1,"""Alice""",null,null
2,"""Bob""",2,25
3,"""Charlie""",3,30


Note that:
- The order of `df1` is maintained in this left join
- The `null` `value` in the last row if `df1` is not joined to the `null` value in `df2`
- If the join column name(s) are not the same in both `DataFrames` then we specify `left_on` and `right_on` instead of `on`
- as we pass `coalesce=False` (which is the default) we get `id` and `id_right` join columns in the output

If we instead pass `coalesce=True` Polars coalesces the join columns `id` and `id_right` into a single `id` column (this was the default behaviour previously, personally this is what I normally want)

请注意：

- 左连接会保持 `df1` 的顺序。

- 如果 `df1` 未与 `df2` 中的 `null` 值连接，则最后一行的 `null` 值将保持不变。

- 如果两个 `DataFrame` 中的连接列名不同，则需要指定 `left_on` 和 `right_on`，而不是 `on`。

- 由于我们传递了 `coalesce=False`（默认值），因此输出中会包含 `id` 和 `id_right` 连接列。

如果我们传递 `coalesce=True`，Polars 会将连接列 `id` 和 `id_right` 合并为一个 `id` 列（这是之前的默认行为，也是我个人通常想要的行为）。

In [8]:
(df1.join(df2, on="id", how="left", coalesce=True))

id,name,age
i64,str,i64
1,"""Alice""",null
2,"""Bob""",25
3,"""Charlie""",30


If we set `join_nulls=True` then Polars does join on `null` values

如果设置 `join_nulls=True`，则 Polars 将基于 `null` 值进行连接。

In [9]:
(df1.join(df2, on="id", how="left", coalesce=False, join_nulls=True))

id,name,id_right,age
i64,str,i64,i64
1,"""Alice""",null,null
2,"""Bob""",2,25
3,"""Charlie""",3,30


When there are duplicate columns in both `DataFrames` Polars adds the suffix `_right` to the duplicate columns on the right by default. We can set an alternative suffix with the `suffix` argument

当两个 `DataFrame` 中存在重复列时，Polars 默认会在右侧的重复列后添加后缀 `_right`。我们可以使用 `suffix` 参数设置其他后缀。

In [10]:
(df1.join(df2, on="id", how="left", coalesce=False, suffix="_right"))

id,name,id_right,age
i64,str,i64,i64
1,"""Alice""",null,null
2,"""Bob""",2,25
3,"""Charlie""",3,30


## Inner joins
In an inner join we only retain the rows in both `DataFrames` where there is a matching join key

## 内连接

在内连接中，我们只保留两个 `DataFrame` 中存在匹配连接键的行。

In [11]:
(df1.join(df2, on="id"))

id,name,age
i64,str,i64
2,"""Bob""",25
3,"""Charlie""",30


As for left joins the output excludes `null` values unless we set `join_nulls=True`

对于左连接，除非我们设置 `join_nulls=True`，否则输出将排除 `null` 值。

In [12]:
(df1.join(df2, on="id", join_nulls=True))

id,name,age
i64,str,i64
2,"""Bob""",25
3,"""Charlie""",30


## Cross join
With a cross join we get the Cartesian product of both tables - so we end up with each row of the left `DataFrame` matched with each row of the right `DataFrame` and there is no join key.

## 交叉连接

通过交叉连接，我们可以得到两个表的笛卡尔积——因此，左侧 `DataFrame` 的每一行都与右侧 `DataFrame` 的每一行相匹配，并且没有连接键。

In [13]:
(df1.join(df2, how="cross"))

id,name,id_right,age
i64,str,i64,i64
1,"""Alice""",2,25
1,"""Alice""",3,30
1,"""Alice""",4,35
2,"""Bob""",2,25
2,"""Bob""",3,30
2,"""Bob""",4,35
3,"""Charlie""",2,25
3,"""Charlie""",3,30
3,"""Charlie""",4,35


## Full outer join
In a full outer join we returns all rows when there is a match in either left or right `DataFrame`

## 全外连接

在全外连接中，如果左侧或右侧 `DataFrame` 中存在匹配项，则返回所有匹配的行。

In [1]:
(df1.join(df2, on="id", how="full"))

NameError: name 'df1' is not defined

In the output we see that:
- we get an `id_right` column as we have the same name for the join column in both `DataFrames`
- the first two rows have the matching keys
- we then get the two rows where there is no match in the left `DataFrame` for `null` and `D`
- we then get the two rows where there is no match in the right `DataFrame` for `null` and `B`

We can of course choose to match on the `null` values with `join_nulls=True`

输出结果显示：

- 由于两个 `DataFrame` 中的连接列名称相同，因此我们得到一个 `id_right` 列。

- 前两行包含匹配的键。

- 接下来是两行，其中 `null` 和 `D` 在左侧 `DataFrame` 中没有匹配项。

- 接下来是两行，其中 `null` 和 `B` 在右侧 `DataFrame` 中没有匹配项。

当然，我们也可以选择使用 `join_nulls=True` 来匹配 `null` 值。

In [16]:
(df1.join(df2, on="id", how="full", join_nulls=True))

id,name,id_right,age
i64,str,i64,i64
2,"""Bob""",2,25
3,"""Charlie""",3,30
null,null,4,35
1,"""Alice""",null,null


## Full outer join with coalesced join keys
An outer-coalesce join is like: 
- an `full outer` join followed by
- a `coalesce` of the `id` and `id_right` columns into a single `id` column with the first non-`null` value

To do this we pass `coalesce=True`

## 带有合并连接键的完全外连接

外合并连接类似于：

- 先执行一个完全外连接，然后

- 将 `id` 和 `id_right` 列合并为一个单独的 `id` 列，合并后的列包含第一个非空值

为此，我们需要传递 `coalesce=True` 参数。


In [18]:
(df1.join(df2, on="id", how="full", coalesce=True))

id,name,age
i64,str,i64
2,"""Bob""",25
3,"""Charlie""",30
4,null,35
1,"""Alice""",null
